# Group 27

# Text Classification


# Dataset-2

# Load the Datasets

In [846]:
import re, string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import Normalizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
import numpy as np
import time
from sklearn.naive_bayes import MultinomialNB

In [824]:
#Second data Set

def Open_Files (path):
    content = []
    file_names = os.listdir(path)
    for file in file_names:
        t = path + file
        content.append(open(t, encoding ='utf-8').read())  #English Text
    return content


In [825]:
import os

training_pos = Open_Files('C:\\Users\\Owner\Documents\\Python\\Assignment2\\Data\\aclImdb_v1\\aclImdb\\train\\pos\\')
training_neg = Open_Files('C:\\Users\\Owner\\Documents\\Python\\Assignment2\\Data\\aclImdb_v1\\aclImdb\\train\\neg\\')
test_pos = Open_Files('C:\\Users\\Owner\\Documents\\Python\\Assignment2\\Data\\aclImdb_v1\\aclImdb\\test\\pos\\')
test_neg = Open_Files ('C:\\Users\\Owner\\Documents\\Python\\Assignment2\\Data\\aclImdb_v1\\aclImdb\\test\\neg\\')

# Data Splitting and Preprocessing

In [845]:
def preprocessing(file):
    text_content = []
    exclude = string.punctuation
    exclude = exclude.replace("-", "")
    pattern = r"[{}]".format(exclude)

    for data in file :
        text = re.sub(r"(<br\s*/><br\s*/>)", " ", str(data))
        text = re.sub(pattern, "", str(text))
        text_content.append(text.lower())
    return text_content

In [833]:
import numpy as np

#Data Spliting and Labelization 
training_CrossValidation = training_pos + training_neg
x_train2 = training_pos[:10000] + training_neg[:10000]
x_train2 = preprocessing(x_train2)
x_val2 = training_pos [10000:] + training_neg[10000:]
x_val2 = preprocessing(x_val2)
x_test2 = test_pos + test_neg
x_test2 = preprocessing(x_test2)
#labelization
y_train2 = np.append(np.ones(10000), np.zeros(10000))
y_val2 = np.append(np.ones(2500), np.zeros(2500))
y_test2 = np.append(np.ones(12500), np.zeros(12500))

# Vectorization

In [829]:
#CountVectorizer (Not Necessary)
vectorizer = CountVectorizer(analyzer='word', stop_words=None, ngram_range=(1,1), binary=True) 

train_set2 = vectorizer.fit_transform(x_train2)
val_set2 = vectorizer.transform(x_val2)
test_set2 = vectorizer.transform(x_test2)

print(train_set2.shape)
print(test_set2.shape)

(20000, 84708)
(25000, 84708)


In [830]:
#TFIDF Vectorizer
tf_idf_vectorizer = TfidfVectorizer()
train_idf2 = tf_idf_vectorizer.fit_transform(x_train2)
val_idf2 = tf_idf_vectorizer.transform(x_val2)
test_idf2 = tf_idf_vectorizer.transform(x_test2)

print(train_idf2.shape)
print(val_idf2.shape)
print(test_idf2.shape)

(20000, 84708)
(5000, 84708)
(25000, 84708)


In [831]:
#Normalization
normalizer_train = Normalizer().fit(X=vectors_train_idf)
train_norm2 = normalizer_train.transform(train_idf2)
val_norm2 = normalizer_train.transform(val_idf2)
test_norm2 = normalizer_train.transform(test_idf2)

print(train_norm2.shape)
print(val_norm2.shape)
print(test_norm2.shape)

(20000, 84708)
(5000, 84708)
(25000, 84708)


# GridSearch for Logistic Regression

In [834]:
clf3 = LogisticRegression(multi_class = 'ovr', solver='saga', max_iter=1000)

tuned_parameters={"C":np.logspace(-3,3,7)}

scores = ['precision', 'recall']

#Tried for the following as well for parameters, L2 was better in this case
#"penalty":["l1","l2"]}#, "dual": ["True", "False"], "max_iter": np.power(10.0, np.arange(-10, 10))}

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf_log2=GridSearchCV(clf3,tuned_parameters,cv=10)
    clf_log2.fit(train_norm2, y_train2)

    t0=time.time()
    print ("training time:", round(time.time()-t0, 3), "s")
    print("Best parameters set found on development set:")
    print()
    print(clf_log2.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf_log2.cv_results_['mean_test_score']
    stds = clf_log2.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf_log2.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    t1=time.time()
    y_pred = clf_log2.predict(val_norm2)
    print ("predict time:", round(time.time()-t1, 3), "s")
    print(metrics.classification_report(y_val2, y_pred, digits = 5))

print("tuned hyperparameters :(best parameters) ",clf_log2.best_params_)
print("accurac y :",clf_log2.best_score_)

# Tuning hyper-parameters for precision

training time: 0.0 s
Best parameters set found on development set:

{'C': 1.0}

Grid scores on development set:

0.769 (+/-0.026) for {'C': 0.001}
0.784 (+/-0.023) for {'C': 0.01}
0.834 (+/-0.017) for {'C': 0.1}
0.865 (+/-0.024) for {'C': 1.0}
0.859 (+/-0.029) for {'C': 10.0}
0.847 (+/-0.029) for {'C': 100.0}
0.841 (+/-0.032) for {'C': 1000.0}

Detailed classification report:

The model is trained on the full development set.
The scores are computed on the full evaluation set.

predict time: 0.016 s
              precision    recall  f1-score   support

         0.0    0.87232   0.88000   0.87614      2500
         1.0    0.87893   0.87120   0.87505      2500

   micro avg    0.87560   0.87560   0.87560      5000
   macro avg    0.87563   0.87560   0.87560      5000
weighted avg    0.87563   0.87560   0.87560      5000

# Tuning hyper-parameters for recall

training time: 0.0 s
Best parameters set found on development set:

{'C': 1.0}

Grid scor

In [837]:
#Optimal Solution
#Val set
clf3 = LogisticRegression(C=1.0,penalty="l2", multi_class = 'ovr', solver='saga', max_iter=1000)
clf3.fit(train_norm2, y_train2)
print ("training time:", round(time.time()-t0, 3), "s")
t2=time.time()
y_pred3 = clf3.predict(val_norm2)
print ("predict time:", round(time.time()-t1, 3), "s")
print("accuracy:")
print(metrics.accuracy_score(y_val2, y_pred3))
print(metrics.classification_report(y_val2, y_pred3))

training time: 263.71 s
predict time: 263.713 s
accuracy:
0.8756
              precision    recall  f1-score   support

         0.0       0.87      0.88      0.88      2500
         1.0       0.88      0.87      0.88      2500

   micro avg       0.88      0.88      0.88      5000
   macro avg       0.88      0.88      0.88      5000
weighted avg       0.88      0.88      0.88      5000



In [839]:
#Optimal Solution
#Test set
t2=time.time()
y_pred3 = clf3.predict(test_norm2)
print ("predict time:", round(time.time()-t2, 3), "s")
print("accuracy:")
print(metrics.accuracy_score(y_test2, y_pred3))
print(metrics.classification_report(y_test2, y_pred3))

predict time: 0.012 s
accuracy:
0.88084
              precision    recall  f1-score   support

         0.0       0.88      0.88      0.88     12500
         1.0       0.88      0.88      0.88     12500

   micro avg       0.88      0.88      0.88     25000
   macro avg       0.88      0.88      0.88     25000
weighted avg       0.88      0.88      0.88     25000



# GridSearch for Multinomial Naive Bayes

In [841]:
n_folds = 10
tuned_parameters = {"alpha":np.logspace(-3,3,7), "fit_prior":["true","false"]}

scores = ['precision', 'recall']

clf4 = MultinomialNB()

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf_nb2=GridSearchCV(clf4,tuned_parameters,cv=10)
    clf_nb2.fit(train_norm2, y_train2)

    t0=time.time()
    print ("training time:", round(time.time()-t0, 3), "s")
    print("Best parameters set found on development set:")
    print()
    print(clf_nb2.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf_nb2.cv_results_['mean_test_score']
    stds = clf_nb2.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf_nb2.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    t1=time.time()
    y_pred4 = clf_nb2.predict(val_norm2)
    print ("predict time:", round(time.time()-t1, 3), "s")
    print(metrics.classification_report(y_val2, y_pred4, digits = 5))

print("tuned hyperparameters :(best parameters) ",clf_nb2.best_params_)
print("accurac y :",clf_nb2.best_score_)

# Tuning hyper-parameters for precision

training time: 0.0 s
Best parameters set found on development set:

{'alpha': 1.0, 'fit_prior': 'true'}

Grid scores on development set:

0.696 (+/-0.067) for {'alpha': 0.001, 'fit_prior': 'true'}
0.696 (+/-0.067) for {'alpha': 0.001, 'fit_prior': 'false'}
0.730 (+/-0.066) for {'alpha': 0.01, 'fit_prior': 'true'}
0.730 (+/-0.066) for {'alpha': 0.01, 'fit_prior': 'false'}
0.773 (+/-0.064) for {'alpha': 0.1, 'fit_prior': 'true'}
0.773 (+/-0.064) for {'alpha': 0.1, 'fit_prior': 'false'}
0.807 (+/-0.041) for {'alpha': 1.0, 'fit_prior': 'true'}
0.807 (+/-0.041) for {'alpha': 1.0, 'fit_prior': 'false'}
0.806 (+/-0.035) for {'alpha': 10.0, 'fit_prior': 'true'}
0.806 (+/-0.035) for {'alpha': 10.0, 'fit_prior': 'false'}
0.747 (+/-0.033) for {'alpha': 100.0, 'fit_prior': 'true'}
0.747 (+/-0.033) for {'alpha': 100.0, 'fit_prior': 'false'}
0.720 (+/-0.036) for {'alpha': 1000.0, 'fit_prior': 'true'}
0.720 (+/-0.036) for {'alpha': 1000.0, 'fit_prior': 'false'

In [843]:
#Optimal Solution
#Validation set
MultinomialNB(alpha="1.0")
clf_nb2.fit(train_norm2, y_train2)
print ("training time:", round(time.time()-t0, 3), "s")
t2=time.time()
y_pred4 = clf_nb2.predict(val_norm2)
print ("predict time:", round(time.time()-t1, 3), "s")
print("accuracy:")
print(metrics.accuracy_score(y_val2, y_pred4))
print(metrics.classification_report(y_val2, y_pred4))

training time: 79.099 s
predict time: 79.103 s
accuracy:
0.8072
              precision    recall  f1-score   support

         0.0       0.77      0.88      0.82      2500
         1.0       0.86      0.73      0.79      2500

   micro avg       0.81      0.81      0.81      5000
   macro avg       0.81      0.81      0.81      5000
weighted avg       0.81      0.81      0.81      5000



In [844]:
#Optimal Solution
#Test set
t2=time.time()
y_pred4 = clf_nb2.predict(test_norm2)
print ("predict time for test:", round(time.time()-t2, 3), "s")
print("accuracy:")
print(metrics.accuracy_score(y_test2, y_pred4))
print(metrics.classification_report(y_test2, y_pred4))

predict time for test: 0.026 s
accuracy:
0.83028
              precision    recall  f1-score   support

         0.0       0.79      0.90      0.84     12500
         1.0       0.88      0.76      0.82     12500

   micro avg       0.83      0.83      0.83     25000
   macro avg       0.84      0.83      0.83     25000
weighted avg       0.84      0.83      0.83     25000

